# Bathroom Ventilation System Analysis

This notebook analyzes the ventilation requirements for a multi-floor facility with several bathrooms. It calculates:
- Required airflow based on room volume and air changes per hour
- Pressure drops in the duct system
- Duct velocity checks
- Appropriate centrifugal fan selection

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import sys
import os

# Import the ventilation calculator module
import ventilation_calculator as vc

## Run Ventilation System Calculations

In [ ]:
# Calculate ventilation system requirements
report = vc.calculate_ventilation_system()

# Print the report
vc.print_report(report)

## Visualize the Results

In [ ]:
# Create visualizations
vc.visualize_system(report)

# Display the saved image
from IPython.display import Image
Image(filename='ventilation_analysis.png')

## Export Results to CSV

In [ ]:
# Export results to CSV files
vc.export_to_csv(report)

# Display the exported CSV files
print("Exported files:")
for file in os.listdir():
    if file.endswith('.csv'):
        print(f"- {file}")

## Detailed Analysis of Airflow Requirements

In [ ]:
# Load and display airflow requirements
airflow_df = pd.read_csv('airflow_requirements.csv')
airflow_df

## Detailed Analysis of Pressure Drops

In [ ]:
# Load and display pressure drops
pressure_df = pd.read_csv('pressure_drops.csv')

# Sort by pressure drop (descending)
pressure_df.sort_values('Pressure Drop (Pa)', ascending=False)

## Fan Recommendations

In [ ]:
# Load and display fan recommendations if file exists
try:
    fan_df = pd.read_csv('fan_recommendations.csv')
    fan_df
except FileNotFoundError:
    print("No suitable fans found. Consider custom solutions or redesigning the duct system.")

## Velocity Issues Analysis

In [ ]:
# Load and display velocity issues if file exists
try:
    velocity_df = pd.read_csv('velocity_issues.csv')
    velocity_df
except FileNotFoundError:
    print("No velocity issues detected.")

## Modify Parameters and Recalculate

You can modify the parameters below
## Modify Parameters and Recalculate

You can modify the parameters below and recalculate the ventilation system requirements.

In [ ]:
# Modify parameters
vc.CHANGES_PER_HOUR = 12  # Air changes per hour
vc.ALTITUDE = 850  # meters above sea level
vc.TEMP_AVG = 25  # °C
vc.RH_AVG = 60  # %

# Recalculate with new parameters
new_report = vc.calculate_ventilation_system()
vc.print_report(new_report)

## Compare Different Air Change Rates

This section compares the effect of different air change rates on the ventilation requirements.

In [ ]:
# Compare different air change rates
air_change_rates = [8, 10, 12, 15]
results = []

for rate in air_change_rates:
    vc.CHANGES_PER_HOUR = rate
    report = vc.calculate_ventilation_system()
    
    for system_name, system_data in report['systems'].items():
        results.append({
            'Air Changes': rate,
            'System': system_name,
            'Total Flow (CFM)': system_data['airflow']['total_cfm'],
            'Total Pressure (inWG)': system_data['pressure_drop']['total_inwg']
        })

# Create DataFrame and plot results
comparison_df = pd.DataFrame(results)
comparison_df

In [ ]:
# Plot comparison of air change rates
plt.figure(figsize=(12, 6))

# Plot airflow vs air changes
plt.subplot(1, 2, 1)
for system in comparison_df['System'].unique():
    system_data = comparison_df[comparison_df['System'] == system]
    plt.plot(system_data['Air Changes'], system_data['Total Flow (CFM)'], 'o-', label=system)
    
plt.xlabel('Air Changes per Hour')
plt.ylabel('Total Airflow (CFM)')
plt.title('Airflow vs Air Changes')
plt.grid(True)
plt.legend()

# Plot pressure vs air changes
plt.subplot(1, 2, 2)
for system in comparison_df['System'].unique():
    system_data = comparison_df[comparison_df['System'] == system]
    plt.plot(system_data['Air Changes'], system_data['Total Pressure (inWG)'], 'o-', label=system)
    
plt.xlabel('Air Changes per Hour')
plt.ylabel('Total Pressure (inWG)')
plt.title('Pressure vs Air Changes')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('air_changes_comparison.png')
plt.show()

## Analyze Extreme Climate Conditions

This section analyzes the ventilation system under extreme climate conditions.

In [ ]:
# Define climate conditions
climate_conditions = [
    {'name': 'Average', 'temp': 25, 'rh': 60},
    {'name': 'Hot & Humid', 'temp': 28, 'rh': 93},
    {'name': 'Cool & Dry', 'temp': 16, 'rh': 30}
]

climate_results = []

for condition in climate_conditions:
    vc.TEMP_AVG = condition['temp']
    vc.RH_AVG = condition['rh']
    vc.CHANGES_PER_HOUR = 12  # Reset to default
    
    report = vc.calculate_ventilation_system()
    
    for system_name, system_data in report['systems'].items():
        climate_results.append({
            'Climate': condition['name'],
            'Temperature (°C)': condition['temp'],
            'Relative Humidity (%)': condition['rh'],
            'System': system_name,
            'Total Flow (CFM)': system_data['airflow']['total_cfm'],
            'Total Pressure (inWG)': system_data['pressure_drop']['total_inwg'],
            'Air Density (kg/m³)': report['systems'][system_name]['airflow']['density'] if 'density' in report['systems'][system_name]['airflow'] else vc.air_density(vc.ALTITUDE, condition['temp'], condition['rh'])
        })

# Create DataFrame and display results
climate_df = pd.DataFrame(climate_results)
climate_df

In [ ]:
# Plot comparison of climate conditions
plt.figure(figsize=(12, 6))

# Plot air density vs climate
plt.subplot(1, 2, 1)
for system in climate_df['System'].unique():
    system_data = climate_df[climate_df['System'] == system]
    plt.bar(system_data['Climate'], system_data['Air Density (kg/m³)'], label=system, alpha=0.7)
    
plt.xlabel('Climate Condition')
plt.ylabel('Air Density (kg/m³)')
plt.title('Air Density vs Climate')
plt.legend()

# Plot pressure vs climate
plt.subplot(1, 2, 2)
for system in climate_df['System'].unique():
    system_data = climate_df[climate_df['System'] == system]
    plt.bar(system_data['Climate'], system_data['Total Pressure (inWG)'], label=system, alpha=0.7)
    
plt.xlabel('Climate Condition')
plt.ylabel('Total Pressure (inWG)')
plt.title('Pressure vs Climate')
plt.legend()

plt.tight_layout()
plt.savefig('climate_comparison.png')
plt.show()

## Conclusion

This analysis has determined the ventilation requirements for the bathroom facilities across multiple floors. The recommended fans should provide adequate ventilation while ensuring that duct velocities remain within acceptable ranges.

### Key Findings:

1. System 1 (PB Baño C, PB Baño D, Cafetín Baño C, Cafetín Baño D) requires a centrifugal fan capable of handling the calculated airflow and static pressure.
2. System 2 (Piso 1 Baño C, Piso 1 Baño D, Piso 2 Baño C, Piso 2 Baño D) requires a separate centrifugal fan with its own specifications.
3. Any velocity issues identified should be addressed by adjusting duct sizes or considering alternative designs.
4. The selected fans provide adequate margin for variations in climate conditions and operational requirements.